In [5]:
import requests
import json
import datetime
import time
import pandas as pd


## Get salary index from Hagstofa Íslands

In [6]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
months = dict(zip(months, [str(_).rjust(2, '0')  for _ in range(1, 13)]))

In [7]:
df_salary = pd.read_csv("https://px.hagstofa.is:443/pxis/sq/a1a47246-889c-4aa1-92e0-b16be1519811", sep=";")\
.rename(columns={"Mánuður": "date", "Vísitölugildi": "value"})\
.assign(date = lambda r: r.date.apply(lambda d: datetime.date(*list(map(int, d.split("M"))), 1)))\
.astype(dict(value = float))\
.astype(dict(date='datetime64[ns]'))

df_salary['value_12m_change'] = df_salary['value'].pct_change(12)


## GET property price index from Þjóðskrá

In [8]:
df_property = pd.read_csv("data/fasteignavisitala.csv")\
.assign(date = lambda r: r['Earliest Date'].pipe(pd.to_datetime).dt.date)\
.rename(columns={
    "Sameinuð vísitala íbúðaverðs": "value"
})\
.loc[:, ['date', 'value']]\
.astype(dict(date='datetime64[ns]'))


## GET Arion Banki index and non-index rates

In [9]:
def transform_arion(df):
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['floating_rate'] = df['floating_rate'].str.replace(',', '.').astype(float)
    df['base_rate'] = df['base_rate'].str.replace(',', '.').astype(float)

    # Extend the dataframe
    arion_date_range = pd.date_range(df['date'].min(), datetime.date.today())
    df = df.set_index('date').reindex(arion_date_range).ffill()

    # Add rate column that is nvl(floating_rate, base_rate)
    df['rate'] = df['floating_rate'].fillna(df['base_rate'])

    return df[['rate']].reset_index().rename(columns={'index': 'date'})


In [10]:
df_arion_index_rate = pd.read_csv("data/downloads/arion-indexed.scv", sep=' ').pipe(transform_arion)

FileNotFoundError: [Errno 2] No such file or directory: 'data/downloads/arion-indexed.scv'

In [15]:
df_arion_nonindex_rate = pd.read_csv("data/downloads/arion-non-indexed.scv", sep=' ').pipe(transform_arion)

In [16]:
df_arion_nonindex_rate\
.merge(df_arion_index_rate, on='date')\
.rename(columns = dict(rate_x='non_index_rate', rate_y='index_rate'))\
.to_csv("data/rates.csv", index=False)

## GET Taxed salaries

Ajust salary index by average salary

Adjust the salary such that it is an indicator of average yearly salary

https://www.ruv.is/frett/2021/07/12/medallaunin-794-thusund-kronur-a-manudi


In [28]:
df_index = df_salary\
.merge(df_property, on='date')\
.rename(columns = dict(
    value_x = 'salary_index',
    value_12m_change_x = 'salary_12m_change',
    value_y = 'property_index',
    value_12m_change_y = 'property_12m_change',
    value = 'cpi_index'
))

salary_factor = 845*1e3/df_index.loc[lambda r: r.date.dt.year == 2024, 'salary_index'].mean()
property_factor = 825e3/df_index.loc[lambda r: r.date == '2025-01-01', 'property_index'].mean()

df_index = df_index\
.assign(
    salary = lambda r: r.salary_index * salary_factor,
    salary_yearly = lambda r: r.salary * 12,
    property = lambda r: r.property_index * property_factor * 120,
    finance_index = lambda r: r.property / r.salary_yearly
)

df_index.to_csv('data/index-data.csv')

In [29]:
def tax_salary(salary, date):

    query = {
        "LaunGreidast": "True",
        "Hjuskaparstada": 1,
        "Tekjuar": date.year,
        "Launamanudur": date.month,
        "Laun": int(salary),
        "LifeyrissjodurHlutfall": "0,04",
        "SereignHlutfall": "0,02",
        "NytingSkattkorts": 100,
        "NytingSkattkortsMaka": 100,
        "UppsafnadurPersonuafslattur": 0,
        "Orlof": 0,
        "Stettarfelag": 0,
        "Annad": 0,
        "MotframlagLifeyrissjodur": 0.0,
        "OkutaekjastyrkurUtan": 0,
        "IdgjaldSlysatryggingSjomanna": 0
    }

    url = "https://reiknivelarws.rsk.is/StadgreidslaForm"
    r = requests.post(url, data=query)
    url_split = r.url.replace(f"{url}/Results?", "").split("&")
    result = {}
    for key, value in [_.split("=") for _ in url_split][:-1]:
        result[key] = value

    result['date'] = date
    result['salary'] = salary

    return result

In [31]:
results = []
for i, row in df_index.loc[lambda r: r.date >= '2015-01-01'].iterrows():
    results.append(tax_salary(row.salary, row.date))
    print(i, row.date)
    time.sleep(1)
df_taxed_salary = pd.DataFrame(results)

312 2015-01-01 00:00:00
313 2015-02-01 00:00:00
314 2015-03-01 00:00:00
315 2015-04-01 00:00:00
316 2015-05-01 00:00:00
317 2015-06-01 00:00:00
318 2015-07-01 00:00:00
319 2015-08-01 00:00:00
320 2015-09-01 00:00:00
321 2015-10-01 00:00:00
322 2015-11-01 00:00:00
323 2015-12-01 00:00:00
324 2016-01-01 00:00:00
325 2016-02-01 00:00:00
326 2016-03-01 00:00:00
327 2016-04-01 00:00:00
328 2016-05-01 00:00:00
329 2016-06-01 00:00:00
330 2016-07-01 00:00:00
331 2016-08-01 00:00:00
332 2016-09-01 00:00:00
333 2016-10-01 00:00:00
334 2016-11-01 00:00:00
335 2016-12-01 00:00:00
336 2017-01-01 00:00:00
337 2017-02-01 00:00:00
338 2017-03-01 00:00:00
339 2017-04-01 00:00:00
340 2017-05-01 00:00:00
341 2017-06-01 00:00:00
342 2017-07-01 00:00:00
343 2017-08-01 00:00:00
344 2017-09-01 00:00:00
345 2017-10-01 00:00:00
346 2017-11-01 00:00:00
347 2017-12-01 00:00:00
348 2018-01-01 00:00:00
349 2018-02-01 00:00:00
350 2018-03-01 00:00:00
351 2018-04-01 00:00:00
352 2018-05-01 00:00:00
353 2018-06-01 0

In [32]:
#df_taxed_salary.to_csv("data/taxed_salary.csv")
df_taxed_salary = pd.DataFrame(results)\
.rename(columns={
    'salary': 'brutto_salary',
    'Lifeyrissjodur': 'mandatory_pension',
    'Sereignarsjodur': 'private_pension',
    'UtborgudLaun': 'netto_salary',
    'GreiddStadgreidsla': 'tax_paid_total',
    'Skattstofn': 'taxable_salary',
    'Personuafslattur': 'personal_discount'
})\
.assign(
    date = lambda r: r.date.pipe(pd.to_datetime)
)\
.set_index('date')\
.loc[:, ['brutto_salary', 'mandatory_pension', 'private_pension', 'netto_salary', 'tax_paid_total', 'taxable_salary', 'personal_discount']]\
.astype(float)

In [33]:
df_taxed_salary.to_csv("data/salary-numbers.csv")

In [34]:
df_taxed_salary

,brutto_salary,mandatory_pension,private_pension,netto_salary,tax_paid_total,taxable_salary,personal_discount
date,,,,,,,
2015-01-01,418974.201576,16759.0,8379.0,295769.0,98066.0,393835.0,50902.0
2015-02-01,421245.292410,16850.0,8424.0,297056.0,98914.0,395970.0,50902.0
2015-03-01,422338.780589,16894.0,8446.0,297675.0,99322.0,396997.0,50902.0
2015-04-01,423264.039818,16931.0,8465.0,298200.0,99668.0,397868.0,50902.0
2015-05-01,425366.901701,17015.0,8507.0,299391.0,100453.0,399844.0,50902.0
...,...,...,...,...,...,...,...
2025-06-01,915838.407300,36634.0,18316.0,633207.0,227680.0,860887.0,68691.0
2025-07-01,913735.545417,36549.0,18274.0,631981.0,226929.0,858910.0,68691.0
2025-08-01,914913.148071,36597.0,18298.0,632668.0,227350.0,860018.0,68691.0
